# Machine Learning Assignment

**Dataset**:       AIRLINE SATISFACTION

**Student ID**:  s5520346  

**Student Name**:  Alex Poulter

## Imports

Add imports here as needed.

Remember to **re-run the cell when you add imports**, so it gets loaded into the virtual notebook environment!

In [130]:
# Data and Datasets
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler

# Clustering
from sklearn.cluster import DBSCAN

# Imputation
from sklearn.impute import KNNImputer

# Validation methods
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold

# Metrics
from sklearn import metrics
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# Classifiers
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier

# Hyper-parameter optimisation
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Feature selection & feature engineering
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.decomposition import PCA

# Stats
from scipy.stats import randint as sp_randint
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import shapiro     # Shapiro Wilk
from scipy.stats import normaltest  # D’Agostino’s K^2
from scipy.stats import anderson    # Anderson-Darling
from scipy.stats import ttest_ind    # independent student t-test; assumes normality
from scipy.stats import mannwhitneyu # non-parametric; doesn't assume normality

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import SVG
from graphviz import Source
from IPython.display import display
from sklearn.tree import export_graphviz

# Utils
import pprint
import numpy as np
from time import time
import openpyxl

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Loading the dataset

In [331]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/airline-satisfaction.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/airline-satisfactionOG.csv')
dftest = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/airline-satisfactiontest.csv')

# printing out dataset info
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12988 entries, 0 to 12987
Data columns (total 24 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   id                                 12988 non-null  int64  
 1   gender                             12988 non-null  object 
 2   customer_type                      12988 non-null  object 
 3   age                                12987 non-null  float64
 4   type_of_travel                     10016 non-null  object 
 5   travel_class                       8466 non-null   object 
 6   flight_distance                    12986 non-null  float64
 7   inflight_wifi_service              8643 non-null   float64
 8   departure_arrival_time_convenient  12988 non-null  int64  
 9   ease_of_online_booking             12988 non-null  int64  
 10  gate_location                      12988 non-null  int64  
 11  food_and_drink                     12988 non-null  int

### describe

In [187]:
print(df.describe())

                  id           age  flight_distance  inflight_wifi_service  \
count   12471.000000  12471.000000     12471.000000           12471.000000   
mean    65648.357950     39.473499      1208.763531               2.721225   
std     37747.560165     15.197827      1004.935709               1.142278   
min        16.000000      7.000000        56.000000               0.000000   
25%     32919.500000     27.000000       416.000000               2.000000   
50%     66027.000000     40.000000       852.000000               2.800000   
75%     98795.500000     51.000000      1773.000000               3.400000   
max    129865.000000     85.000000      4983.000000               5.000000   

       departure_arrival_time_convenient  ease_of_online_booking  \
count                       12471.000000            12471.000000   
mean                            3.036485                2.739636   
std                             1.532340                1.395364   
min                      

In [188]:
df.head()

,id,age,flight_distance,inflight_wifi_service,departure_arrival_time_convenient,ease_of_online_booking,gate_location,food_and_drink,online_boarding,seat_comfort,...,checkin_service,inflight_service,cleanliness,departure_delay_in_minutes,arrival_delay_in_minutes,class_encoded,gender_encoded,customer_type_encoded,type_of_travel_encoded,travel_class_encoded
0,69571.0,30.0,2586.0,2.0,2.0,2.0,4.0,2.0,4.0,4.0,...,4.0,2.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0
1,19953.0,66.0,315.0,2.4,4.0,3.0,1.0,5.0,2.2,5.0,...,4.0,4.0,4.0,0.0,26.0,1.0,0.0,1.0,1.0,0.0
2,8913.0,39.0,510.0,3.4,5.0,5.0,1.0,4.0,5.0,4.0,...,5.0,4.0,4.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0
3,7655.0,21.0,767.0,4.0,0.0,4.0,3.0,4.0,4.0,4.0,...,3.0,4.0,4.0,15.0,0.0,2.0,0.0,0.0,0.0,1.0
4,100573.0,53.0,486.0,2.2,1.0,1.0,1.0,5.0,4.0,4.0,...,3.0,5.0,4.0,0.0,0.0,2.0,0.0,1.0,0.0,3.0


## Exploratory data analysis

Section for exploratory data analysis, to address **Tasks 1.1 and 1.2**.

**OBS:** You may need to do some data cleaning before you do your full exploratory data analysis, though you will find that some functions we'll cover in this unit are able to handle things like missing values and non-numeric data.

**Create more cells as needed!**

In [304]:
df['gender'].value_counts().plot(kind='pie', title='Gender distribution');

KeyError: 'gender'

In [ ]:
sns.countplot(data=df, x='class')
plt.title('Distribution of Customer Satisfaction')
plt.xlabel('Satisfaction Level')
plt.ylabel('Count')
plt.show()

In [ ]:
sns.countplot(data=df, x='customer_type')
plt.title('Distribution of Customer Satisfaction')
plt.xlabel('Customer')
plt.ylabel('Count')
plt.show()

In [ ]:
sns.countplot(df, x="gender", hue="class")

In [ ]:
sns.countplot(df, x="customer_type", hue="class")

In [ ]:
sns.countplot(df, x="type_of_travel", hue="travel_class")

In [ ]:
sns.displot(data=df, x="age", hue="gender", kind="kde")

In [ ]:
sns.violinplot(data=df, x="class", y="age", hue="gender", split=True, inner="quart")

In [ ]:
sns.violinplot(data=df, x="class", y="flight_distance", hue="gender", split=True, inner="quart")

In [ ]:
sns.violinplot(data=df, x="class", y="age", hue="customer_type", split=True, inner="quart")

In [ ]:
plt.figure();

# setting up custom colouring
color = {'boxes': 'DarkGreen', 'whiskers': 'DarkOrange', 'medians': 'DarkBlue', 'caps': 'Gray'}
columns_to_plot = ['departure_delay_in_minutes', 'arrival_delay_in_minutes']
# a box plot for all features
box_plot = df[columns_to_plot].plot.box(color=color, sym='r+');
box_plot.set_title('Box plot for Departure and Arrival delays')
box_plot.set_ylabel('minutes')
box_plot.set_ylim([0, 400])

In [ ]:
plt.figure();

# setting up custom colouring
color = {'boxes': 'DarkGreen', 'whiskers': 'DarkOrange', 'medians': 'DarkBlue', 'caps': 'Gray'}
columns_to_plot = ['inflight_wifi_service', 'departure_arrival_time_convenient', 'ease_of_online_booking', 'gate_location', 'food_and_drink', 'online_boarding', 'seat_comfort', 'inflight_entertainment', 'on_board_service', 'leg_room_service', 'baggage_handling', 'checkin_service', 'inflight_service', 'cleanliness']
# a box plot for all features
box_plot = df[columns_to_plot].plot.box(color=color, sym='r+');
box_plot.set_title('Box plot')
box_plot.set_ylabel('Customer rating')
box_plot.set_ylim([-1, 6])
box_plot.set_xticklabels(box_plot.get_xticklabels(), rotation=90)

In [ ]:
sns.scatterplot(x="arrival_delay_in_minutes", y="departure_delay_in_minutes", hue="class", data=df, palette="bright", s=20, alpha=0.8)

plt.xlabel('arrival delay in minutes')
plt.ylabel('departure delay in minutes')

plt.xlim(0, 500)
plt.ylim(0, 500)

plt.show()

In [ ]:
sns.scatterplot(x="age", y="flight_distance", hue="travel_class", data=df, palette="bright", s=20, alpha=0.8)

plt.xlabel('age')
plt.ylabel('flight distance')

plt.xlim(0, 100)
plt.ylim(0, 4200)

plt.show()

In [ ]:
# Scatter plot
sns.scatterplot(x="age", y="flight_distance", data=df, s=20, alpha=0.8)
plt.xlabel('age')
plt.ylabel('flight distance')
plt.xlim(0, 100)
plt.ylim(0, 4500)

# Heatmap
x = df['age']
y = df['flight_distance']

# Drop NaN values
xy_valid = pd.concat([x, y], axis=1).dropna()

heatmap, xedges, yedges = np.histogram2d(xy_valid['age'], xy_valid['flight_distance'], bins=(50, 50))
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

plt.clf()
plt.imshow(heatmap.T, extent=extent, origin='lower', aspect='auto', cmap='viridis')  # You can choose a different colormap
plt.colorbar(label='Frequency')
plt.xlabel('age')
plt.ylabel('flight distance')
plt.show()

In [ ]:
# Scatter plot
sns.scatterplot(x="age", y="flight_distance", hue="class", data=df, palette="bright", s=20, alpha=0.8)
plt.xlabel('age')
plt.ylabel('flight distance')
plt.xlim(0, 100)
plt.ylim(0, 4200)

# Create separate heatmaps for each class
classes = df['class'].unique()

for class_label in classes:
    class_df = df[df['class'] == class_label]

    # Heatmap
    x = class_df['age']
    y = class_df['flight_distance']

    # Drop NaN values
    xy_valid = pd.concat([x, y], axis=1).dropna()

    heatmap, xedges, yedges = np.histogram2d(xy_valid['age'], xy_valid['flight_distance'], bins=(50, 50))
    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

    plt.clf()
    plt.imshow(heatmap.T, extent=extent, origin='lower', aspect='auto', cmap='viridis')
    plt.colorbar(label='Frequency')
    plt.xlabel('age')
    plt.ylabel('flight distance')
    plt.title(f'Heatmap for Class: {class_label}')
    plt.show()

In [ ]:
# Changes the markers for non-subs and plagiarism
fig = plt.figure()
fig, ax = plt.subplots()
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling

# group the data instances according to the different classes they belong to
groups = df.groupby('class')

# iterate over each group and plot (scatter)
for name, group in groups:
    ax.plot(group["flight_distance"], group["arrival_delay_in_minutes"], marker='.', linestyle='', ms=12, label=name, alpha=0.2)

# set labels for the y and x axes
ax.set_xlabel('delay')
ax.set_ylabel('distance')

# set the value ranges for the axes
ax.set_xlim(0,4000)
ax.set_ylim(0,200)


In [ ]:
# Changes the markers for non-subs and plagiarism
fig = plt.figure()
fig, ax = plt.subplots()
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling

# group the data instances according to the different classes they belong to
groups = df.groupby('class')

# iterate over each group and plot (scatter)
for name, group in groups:
    ax.plot(group["arrival_delay_in_minutes"], group["flight_distance"], marker='.', linestyle='', ms=12, label=name)

# set labels for the y and x axes
ax.set_xlabel('delay')
ax.set_ylabel('distance')

# set the value ranges for the axes
ax.set_xlim(0,500)
ax.set_ylim(0,4200)


In [ ]:
# Changes the markers for non-subs and plagiarism
fig = plt.figure()
fig, ax = plt.subplots()
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling

# group the data instances according to the different classes they belong to
groups = df.groupby('class')

# iterate over each group and plot (scatter)
for name, group in groups:
    ax.plot(group["departure_delay_in_minutes"], group["flight_distance"], marker='.', linestyle='', ms=12, label=name)

# set labels for the y and x axes
ax.set_xlabel('delay')
ax.set_ylabel('distance')

# set the value ranges for the axes
ax.set_xlim(0,500)
ax.set_ylim(0,4200)

In [ ]:
corr = df.corr(method='pearson')
corr = df.corr(method='spearman')

In [ ]:
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(corr, dtype=bool))
ax = sns.heatmap(
    corr,
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True,
    annot=True,
    fmt=".2f",
    linewidths=.5,
    mask=mask
)

plt.show()

In [ ]:
sns.pairplot(data=df, hue="class")

In [ ]:
sns.pairplot(data=df, hue="travel_class")

In [ ]:
sns.pairplot(data=df, hue="customer_type")

In [ ]:
def plot_cluster_graph(X, labels, core_samples_mask, algorithm_name, n_clusters):
  X_df = df[["departure_delay_in_minutes", "flight_distance"]]
  X_df = StandardScaler().fit_transform(X_df)

  db = DBSCAN(eps=0.5, min_samples=10).fit(X_df)
  labels = db.labels_

# Number of clusters in labels, ignoring noise (if present)
  n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
  n_noise_ = list(labels).count(-1)

# Create a mask for visualising the core samples in each cluster
# - First creates a NumPy array of False entries
# - Then sets the entries for the core instances to True
  core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
  core_samples_mask[db.core_sample_indices_] = True

# Printing some stats
# can't do all stats as above, as we don't have a ground truth of what clusters
# each data point should belong to as in the test data
  print('Estimated number of clusters: %d' % n_clusters_)
  print('Estimated number of noise points: %d' % n_noise_)

# Plotting the clustering outputs
plot_cluster_graph(X_df, labels, core_samples_mask, 'DBSCAN', n_clusters_)

In [ ]:
X_df = df[["departure_delay_in_minutes", "flight_distance"]]
X_df = StandardScaler().fit_transform(X_df)

# Calculate DBSCAN clustering results
db = DBSCAN(eps=0.5, min_samples=10).fit(X_df)
labels = db.labels_

# Number of clusters in labels, ignoring noise (if present)
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

# Create a mask for visualizing the core samples in each cluster
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True

# Printing some stats
print('Estimated number of clusters: %d' % n_clusters_)

plt.scatter(X_df[:, 0], X_df[:, 1], c=labels, cmap='autumn', s=50, alpha=0.8)
plt.title('DBSCAN Clustering')
plt.xlabel('Departure Delay')
plt.ylabel('Flight Distance')
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
X = pd.DataFrame(df, columns=['id', 'age', 'flight_distance', 'inflight_wifi_service', 'departure_arrival_time_convenient', 'ease_of_online_booking',
                              'inflight_entertainment', 'on_board_service', 'leg_room_service', 'baggage_handling', 'checkin_service', 'inflight_service', 'cleanliness', 'departure_delay_in_minutes',
                              'arrival_delay_in_minutes'])
y = df['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=12)

In [ ]:
rf = RandomForestRegressor(n_estimators=100)
rf.fit(X_train, y_train)

In [ ]:
print(df.head())

In [ ]:
X = df.drop('class_satisfied, class_neutral', axis=1)  # Replace 'target_column' with the actual name of your target column
y = df['class_neutral, class_satisfied']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

feature_names = X.columns.tolist()
forest = RandomForestClassifier(random_state=0)
forest.fit(X_train, y_train)

start_time = time.time()
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
elapsed_time = time.time() - start_time

print(f"Elapsed time to compute the importances: {elapsed_time:.3f} seconds")

forest_importances = pd.Series(importances, index=feature_names)

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()
plt.show()

In [ ]:
df.isnull().sum()

In [ ]:
df.isnull().sum(axis=1).value_counts()

## Core data pre-processing
Do core data pre-processing here, which is not part of your machine learning experiments.

For example, this may be to convert non-numeric features into numeric ones.

In [321]:
values = {"arrival_delay_in_minutes": 0, "departure_delay_in_minutes": 0}
df.fillna(value=values, inplace=True)

In [333]:
threshold_count = 4

# The the 'null' count for every row
df_row_null_count = df.isnull().sum(axis=1)

# Drop any rows that reach the 'threshold_count' for 'null' values
df = df.drop(df_row_null_count[df_row_null_count >= threshold_count].index, axis=0)

In [334]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder(categories=[["Female", "Male"]])
gender_column = df[['gender']].values.reshape(-1, 1)
df['gender_encoded'] = encoder.fit_transform(gender_column)

df.drop(['gender'], axis=1, inplace=True)

encoder = OrdinalEncoder(categories=[["disloyal Customer", "Loyal Customer"]])
customer_type_column = df[['customer_type']].values.reshape(-1, 1)
df['customer_type_encoded'] = encoder.fit_transform(customer_type_column)

df.drop(['customer_type'], axis=1, inplace=True)

df['type_of_travel'] = df['type_of_travel'].fillna('Unknown')
encoder = OrdinalEncoder(categories=[["Business travel", "Personal Travel", "Unknown"]])
type_of_travel_column = df[['type_of_travel']].values.reshape(-1, 1)
df['type_of_travel_encoded'] = encoder.fit_transform(type_of_travel_column)

df.drop(['type_of_travel'], axis=1, inplace=True)

df['travel_class'] = df['travel_class'].fillna('Unknown')
encoder = OrdinalEncoder(categories=[["Eco", "Business", "Eco Plus", "Unknown"]])
travel_class_column = df[['travel_class']].values.reshape(-1, 1)
df['travel_class_encoded'] = encoder.fit_transform(travel_class_column)

df.drop(['travel_class'], axis=1, inplace=True)

encoder = OrdinalEncoder(categories=[["dissatisfied", "neutral", "satisfied"]])
class_column = df[['class']].values.reshape(-1, 1)
df['class_encoded'] = encoder.fit_transform(class_column)

df.drop(['class'], axis=1, inplace=True)

print(df)

           id   age  flight_distance  inflight_wifi_service  \
0       69571  30.0           2586.0                    2.0   
2        8913  39.0            510.0                    NaN   
3        7655  21.0            767.0                    4.0   
4      100573  53.0            486.0                    NaN   
6       98573  47.0           3221.0                    2.0   
...       ...   ...              ...                    ...   
12982   53119  54.0            565.0                    2.0   
12983   35298  61.0            944.0                    NaN   
12984  108028  44.0           1508.0                    NaN   
12986   64469  21.0            469.0                    NaN   
12987   35804  37.0            937.0                    1.0   

       departure_arrival_time_convenient  ease_of_online_booking  \
0                                      2                       2   
2                                      5                       5   
3                                      

In [175]:
# cat_variables = df[['gender', 'customer_type', 'type_of_travel', 'travel_class', 'class']]
# cat_dummies = pd.get_dummies(cat_variables, drop_first=True)

# df = df.drop(['gender', 'customer_type', 'type_of_travel', 'travel_class', 'class'], axis=1)
# df = pd.concat([df, cat_dummies], axis=1)

# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# df = pd.DataFrame(scaler.fit_transform(df), columns = df.columns)

In [179]:
# from sklearn.preprocessing import MinMaxScaler
# # Corrected Sample DataFrame with varying lengths
# data = {'gender': ['Male', 'Female', np.nan],
#         'customer_type': ['Loyal Customer', 'disloyal Customer', np.nan],
#         'type_of_travel': ['Business travel', 'Personal Travel', np.nan],
#         'travel_class': ['Business', 'Eco', 'Eco Plus'],
#         'class': ['satisfied', 'neutral', 'dissatisfied']}
# df = pd.DataFrame(data)

# # Convert categorical variables to dummy variables
# cat_dummies = pd.get_dummies(df[['gender', 'customer_type', 'type_of_travel', 'travel_class', 'class']])

# # Combine binary dummy variables for each category
# df['gender_combined'] = cat_dummies[['gender_Male', 'gender_Female']].sum(axis=1)
# df['customer_type_combined'] = cat_dummies[['customer_type_Loyal Customer', 'customer_type_disloyal Customer']].sum(axis=1)
# df['type_of_travel_combined'] = cat_dummies[['type_of_travel_Business travel', 'type_of_travel_Personal Travel']].sum(axis=1)
# df['travel_class_combined'] = cat_dummies[['travel_class_Business', 'travel_class_Eco', 'travel_class_Eco Plus']].sum(axis=1)
# df['class_combined'] = cat_dummies[['class_satisfied', 'class_neutral', 'class_dissatisfied']].sum(axis=1)

# # Drop the original categorical columns if needed
# df = df.drop(['gender', 'customer_type', 'type_of_travel', 'travel_class', 'class'], axis=1)

# # Normalize the remaining numerical columns
# scaler = MinMaxScaler()
# df[df.columns] = scaler.fit_transform(df)

# print(df)

In [1]:
# df_mean = df.fillna(df.mean())
# df_mean.info()

In [2]:
# df_median = df.fillna(df.median())
# df_median.info()

In [335]:
imputer = KNNImputer(n_neighbors=5)
df = pd.DataFrame(imputer.fit_transform(df),columns = df.columns)

In [185]:
print(df.head())

         id   age  flight_distance  inflight_wifi_service  \
0   69571.0  30.0           2586.0                    2.0   
1   19953.0  66.0            315.0                    2.4   
2    8913.0  39.0            510.0                    3.4   
3    7655.0  21.0            767.0                    4.0   
4  100573.0  53.0            486.0                    2.2   

   departure_arrival_time_convenient  ease_of_online_booking  gate_location  \
0                                2.0                     2.0            4.0   
1                                4.0                     3.0            1.0   
2                                5.0                     5.0            1.0   
3                                0.0                     4.0            3.0   
4                                1.0                     1.0            1.0   

   food_and_drink  online_boarding  seat_comfort  ...  checkin_service  \
0             2.0              4.0           4.0  ...              4.0   
1         

## Task 2.1 - ML Workflow to Critically Evaluate

In [235]:
# Dropping all rows with missing values
df2 = df2.dropna()

# Converting all non-numeric (object) features to numeric
cat_columns = df2.select_dtypes(['object']).columns
df2[cat_columns] = df2[cat_columns].apply(lambda x: x.astype('category')) # converting 'object' columns to 'category' type
df2[cat_columns] = df2[cat_columns].apply(lambda x: x.cat.codes) # converting the 'category' columns to integer encoded values
print(df2["class"].value_counts())

# Splits the Pandas DataFrame into a feature matrix (X) and class/label vector (y)
X = df2.iloc[:,:len(df2.columns)-1]
y = df2.iloc[:,len(df2.columns)-1]

# Splitting dataset for hold-out validation
X_train, X_test, y_train, y_test = train_test_split(X, # feature matrix
                                                    y, # label vector
                                                    test_size=0.2,
                                                    random_state=1,
                                                    stratify=None
                                                   )

1    354
2    261
0     57
Name: class, dtype: int64


In [237]:
# Creating and testing a Logistic Regression Model
model = LogisticRegression()

# Training the model
model.fit(X_train, y_train)

# Testing the model
y_pred = model.predict(X_test)

# Printing out performance of the model
print("Accuracy: %s" % (metrics.accuracy_score(y_test, y_pred)*100))
print(metrics.confusion_matrix(y_test, y_pred))

Accuracy: 54.074074074074076
[[ 0 11  4]
 [ 0 52 13]
 [ 0 34 21]]


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Task 2.3 - Evaluation of Improved ML Workflow

Add code for running your **improved** machine learning experiments below.


In [325]:
X = df.iloc[:,:len(df.columns)-1]
y = df.iloc[:,len(df.columns)-1]

In [326]:
print(X.shape)
print(y.shape)

(12471, 23)
(12471,)


In [337]:
# Instantiating model
model = KNeighborsClassifier(n_neighbors=10)

# Hold-out validation - default 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None)

y_train_flat = y_train.values.ravel()

# Training the model
model.fit(X_train, y_train_flat)

# Testing the model
y_pred = model.predict(X_test)

# Printing out confusion matrix and accuracy
print(metrics.confusion_matrix(y_test, y_pred))
print("\nAccuracy (Testing):  %0.2f " % (metrics.accuracy_score(y_test, y_pred)))

[[  0 112  29]
 [  0 951 322]
 [  4 532 545]]

Accuracy (Testing):  0.60 


In [338]:
model = KNeighborsClassifier(n_neighbors=10)

# cross-validation of the model
scores = cross_val_score(model, X, y, cv=10)

# Printing results
print("Accuracy:  %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy:  0.60 (+/- 0.02)


In [300]:
# Visualising the Decision Tree
def visualise_tree(model, feature_names, class_labels):
    """
    A function to visualise a decision tree model. This requires graphviz to be installed.
    :param model: the decision tree model
    :param feature_list: a list of names for the features
    :param class_labels: a list of names for the class labels
    """
    graph = Source(tree.export_graphviz(model, out_file=None
       , feature_names=feature_names, class_names=class_labels
       , filled = True))
    graph_svg = SVG(graph.pipe(format='svg'))
    display(graph_svg)

In [339]:
# Instantiating model
model = DecisionTreeClassifier(random_state=None,
                               max_depth=9, # smaller or larger depth?
                               criterion='entropy') # gini vs entropy?)

# defining multiple metrics for scoring
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'prec_macro': make_scorer(precision_score, average='macro', zero_division=0),
    'rec_macro': make_scorer(recall_score, average='macro', zero_division=0)
}

# cross-validation of the model
scores = cross_validate(model, X, y, cv=10, scoring=scoring, return_train_score=True, return_estimator=True)

# Printing results
print("Accuracy:  %0.2f (+/- %0.2f)" % (scores['test_accuracy'].mean(), scores['test_accuracy'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores['test_prec_macro'].mean(), scores['test_prec_macro'].std() * 2))
print("Recall:    %0.2f (+/- %0.2f)" % (scores['test_rec_macro'].mean(), scores['test_rec_macro'].std() * 2))

Accuracy:  0.83 (+/- 0.01)
Precision: 0.60 (+/- 0.20)
Recall:    0.58 (+/- 0.01)


In [295]:
def report(results, rank_metric='score', n_top=3):
    """
    Utility function to report best scores.
    :param results: the cv_results_ data structure from the optimisation algorithm
    :param rank_metric: name of the metric to report results for
    :param n_top: the number of top results to report
    """
    print("\nModels ranked according to", rank_metric)
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results["rank_test_" + rank_metric] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.2f} (+/- {1:.2f})".format(
                  results["mean_test_" + rank_metric][candidate],
                  results["std_test_" + rank_metric][candidate]*2))
            print("Params: {0}".format(results['params'][candidate]))
            print("")

In [340]:
# instantiating the model
model = DecisionTreeClassifier()

# defining parameter grid
param_grid = {"max_depth": [2, 4, 6, 8, None],
              "max_features": [2, 4, 6, 8, 10],
              "min_samples_split": [2, 4, 6, 8, 10],
              "criterion": ["gini", "entropy"],
              "splitter": ["best", "random"]}

# run grid search
print("\n> STARTING GRID SEARCH ...")
grid_search = GridSearchCV(model, param_grid=param_grid, cv=n_folds)

start_time = time()
grid_search.fit(X, y)
end_time = time()

print("> GRID SEARCH COMPLETE")

print("\nGridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (end_time - start_time, len(grid_search.cv_results_['params'])))
report(grid_search.cv_results_)


> STARTING GRID SEARCH ...
> GRID SEARCH COMPLETE

GridSearchCV took 112.34 seconds for 500 candidate parameter settings.

Models ranked according to score
Model with rank: 1
Mean validation score: 0.82 (+/- 0.03)
Params: {'criterion': 'gini', 'max_depth': 8, 'max_features': 10, 'min_samples_split': 8, 'splitter': 'best'}

Model with rank: 2
Mean validation score: 0.82 (+/- 0.02)
Params: {'criterion': 'entropy', 'max_depth': 8, 'max_features': 10, 'min_samples_split': 10, 'splitter': 'best'}

Model with rank: 3
Mean validation score: 0.82 (+/- 0.02)
Params: {'criterion': 'gini', 'max_depth': 8, 'max_features': 6, 'min_samples_split': 8, 'splitter': 'best'}



In [255]:
# Instantiating model
model = RandomForestClassifier(n_estimators=100)

# Hold-out validation - default 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None)

y_train_flat = y_train.values.ravel()

# Training the model
model.fit(X_train, y_train_flat)

# Testing the model
y_pred = model.predict(X_test)

# Printing out confusion matrix and accuracy
print(metrics.confusion_matrix(y_test, y_pred))
print("\nAccuracy (Testing):  %0.2f " % (metrics.accuracy_score(y_test, y_pred)))

[[   0  140    2]
 [   0 1221   65]
 [   0  120  947]]

Accuracy (Testing):  0.87 


In [265]:
# Assuming X is a Pandas DataFrame
X = X.values if isinstance(X, pd.DataFrame) else X

# Instantiating model
model = RandomForestClassifier(n_estimators=100)

# arrays for storing performance metrics; accuracy, precision, and recall
acc_list = np.array([])
prec_list = np.array([])
rec_list = np.array([])

# arrays for storing the test data + predictions
# this will allow us to do a confusion matrix based on all folds in the end
y_test_aggregated = np.array([])
y_pred_aggregated = np.array([])

# creating/setting up the folds
cv = StratifiedKFold(n_splits=10)
cv.get_n_splits(X, y)

# cross-validation of the model
for i, (train_index, test_index) in enumerate(cv.split(X, y)):
    # Printing out progress (which fold it is)
    print(f">>> Fold {i}..")

    # Getting data for each fold (as the above returns the indices of the data points selected for training and testing)
    X_train, y_train, X_test, y_test = X[train_index], y[train_index], X[test_index], y[test_index]

    # Training the model
    model.fit(X_train, y_train)

    # Testing the model
    y_pred = model.predict(X_test)

    # Appending performance metrics to arrays created above
    acc_list = np.append(acc_list, accuracy_score(y_test, y_pred))
    prec_list = np.append(prec_list, precision_score(y_test, y_pred, average='macro', zero_division=0))
    rec_list = np.append(rec_list, recall_score(y_test, y_pred, average='macro'))

    # Appending the test data and predictions to do an aggregated confusion matrix at the end
    y_test_aggregated = np.append(y_test_aggregated, y_test)
    y_pred_aggregated = np.append(y_pred_aggregated, y_pred)

    # Printing out confusion matrix and accuracy
    print("\nAccuracy (Testing):  %0.2f \n" % (accuracy_score(y_test, y_pred)))
    print(confusion_matrix(y_test, y_pred))
    print()

# Printing results
print("\n>>> Average performance stats (from all folds)\n")
print("Accuracy:  %0.2f (+/- %0.2f)" % (acc_list.mean(), acc_list.std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (prec_list.mean(), prec_list.std() * 2))
print("Recall:    %0.2f (+/- %0.2f)" % (rec_list.mean(), rec_list.std() * 2))
print()
print(confusion_matrix(y_test_aggregated, y_pred_aggregated))


>>> Fold 0..

Accuracy (Testing):  0.86 

[[  0  64   7]
 [  0 595  39]
 [  0  64 479]]

>>> Fold 1..

Accuracy (Testing):  0.87 

[[  0  63   7]
 [  0 607  27]
 [  0  68 475]]

>>> Fold 2..

Accuracy (Testing):  0.88 

[[  0  67   3]
 [  0 602  32]
 [  0  45 498]]

>>> Fold 3..

Accuracy (Testing):  0.88 

[[  0  70   0]
 [  0 604  30]
 [  0  51 492]]

>>> Fold 4..

Accuracy (Testing):  0.87 

[[  0  68   2]
 [  0 600  34]
 [  0  61 482]]

>>> Fold 5..

Accuracy (Testing):  0.87 

[[  0  66   4]
 [  0 607  27]
 [  0  67 476]]

>>> Fold 6..

Accuracy (Testing):  0.88 

[[  0  67   4]
 [  0 601  32]
 [  0  47 496]]

>>> Fold 7..

Accuracy (Testing):  0.86 

[[  0  69   2]
 [  0 595  38]
 [  0  71 472]]

>>> Fold 8..

Accuracy (Testing):  0.87 

[[  0  66   5]
 [  0 605  28]
 [  0  64 479]]

>>> Fold 9..

Accuracy (Testing):  0.87 

[[  0  68   3]
 [  0 602  31]
 [  0  65 478]]


>>> Average performance stats (from all folds)

Accuracy:  0.87 (+/- 0.02)
Precision: 0.59 (+/- 0.01)
Recall:

In [306]:
from sklearn.preprocessing import Normalizer

normalizer = Normalizer()
normalizer.fit(X)
X_normalised = normalizer.transform(X)
# X_normalised

In [313]:
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

In [316]:
epochs = 5000
model = MLPClassifier(hidden_layer_sizes=(50), max_iter=epochs)

scores = cross_validate(model, X_normalised, y, cv=5)
print("Accuracy:          %0.2f (+/- %0.2f)" % (scores['test_score'].mean(), scores['test_score'].std() * 2))
print("Training time (s): %0.2f (+/- %0.2f)" % (scores['fit_time'].mean(), scores['fit_time'].std() * 2))
print("Testing time (s):  %0.2f (+/- %0.2f)" % (scores['score_time'].mean(), scores['score_time'].std() * 2))

Accuracy:          0.58 (+/- 0.02)
Training time (s): 11.15 (+/- 8.55)
Testing time (s):  0.01 (+/- 0.01)


In [314]:
epochs = 5000
model = MLPClassifier(hidden_layer_sizes=(30), max_iter=epochs)

scores = cross_validate(model, X_scaled, y, cv=5)
print("Accuracy:          %0.2f (+/- %0.2f)" % (scores['test_score'].mean(), scores['test_score'].std() * 2))
print("Training time (s): %0.2f (+/- %0.2f)" % (scores['fit_time'].mean(), scores['fit_time'].std() * 2))
print("Testing time (s):  %0.2f (+/- %0.2f)" % (scores['score_time'].mean(), scores['score_time'].std() * 2))

Accuracy:          0.84 (+/- 0.00)
Training time (s): 14.72 (+/- 8.62)
Testing time (s):  0.01 (+/- 0.01)


In [310]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(X_normalised, y, test_size=0.4, random_state=None, stratify=y)

model = MLPClassifier(hidden_layer_sizes=(10))
model = model.fit(X_train,y_train)
y_pred = model.predict(X_test)

print("Accuracy", metrics.accuracy_score(y_test, y_pred))
print("\n", metrics.confusion_matrix(y_test, y_pred))
print("\n", metrics.classification_report(y_test, y_pred))

Accuracy 0.5628382441371016

 [[   0  199   83]
 [   0 1870  665]
 [   0 1234  938]]

               precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       282
         1.0       0.57      0.74      0.64      2535
         2.0       0.56      0.43      0.49      2172

    accuracy                           0.56      4989
   macro avg       0.37      0.39      0.38      4989
weighted avg       0.53      0.56      0.54      4989



/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and 